In [1]:
import pandas as pd
import numpy as np
import os
import time
import datetime
import requests
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from haversine import haversine, Unit
from geopy.geocoders import Nominatim
from tenacity import retry, wait_fixed, stop_after_attempt
import requests_cache
import openmeteo_requests
from openmeteo_sdk import Variable

from sklearn.model_selection import train_test_split

In [2]:
def filter_csvs(folder_path, heart_rate_col, cadence_col):
    valid_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            if heart_rate_col in df.columns and cadence_col in df.columns:
                if not df[heart_rate_col].isnull().any() and not df[cadence_col].isnull().any():
                    valid_files.append(folder_path + '/' + filename)
        else:
          pass

    return valid_files

In [3]:
folder_path = "./ReeceCSV"
heart_rate_column = "Heart Rate"
cadence_column = "Cadence"

csvs = filter_csvs(folder_path, heart_rate_column, cadence_column)

In [4]:
print(csvs)

['./ReeceCSV/activity_13226796543.csv', './ReeceCSV/activity_13240047262.csv', './ReeceCSV/activity_13254761903.csv', './ReeceCSV/activity_13268407265.csv', './ReeceCSV/activity_13278145933.csv', './ReeceCSV/activity_13304138921.csv', './ReeceCSV/activity_13357872606.csv', './ReeceCSV/activity_13362658383.csv', './ReeceCSV/activity_13376823367.csv', './ReeceCSV/activity_13406115720.csv', './ReeceCSV/activity_13422875792.csv', './ReeceCSV/activity_13452548822.csv', './ReeceCSV/activity_13464219927.csv', './ReeceCSV/activity_13595180027.csv', './ReeceCSV/activity_13604744119.csv', './ReeceCSV/activity_13678339444.csv', './ReeceCSV/activity_13678341550.csv', './ReeceCSV/activity_13751786621.csv', './ReeceCSV/activity_13758815843.csv', './ReeceCSV/activity_13784603582.csv', './ReeceCSV/activity_13801311250.csv', './ReeceCSV/activity_13833774321.csv', './ReeceCSV/activity_13849441777.csv', './ReeceCSV/activity_13879283945.csv', './ReeceCSV/activity_13906330417.csv', './ReeceCSV/activity_139

In [5]:
dfs = [pd.read_csv(csv_file) for csv_file in csvs]

In [6]:
from openmeteo_sdk.Variable import Variable

om = openmeteo_requests.Client()
params = {
    "latitude": 52.54,
    "longitude": 13.41,
    "hourly": ["temperature_2m", "precipitation", "wind_speed_10m"],
    "current": ["temperature_2m", "relative_humidity_2m"]
}

responses = om.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values
current = response.Current()
current_variables = list(map(lambda i: current.Variables(i), range(0, current.VariablesLength())))
current_temperature_2m = next(filter(lambda x: x.Variable() == Variable.temperature and x.Altitude() == 2, current_variables))
current_relative_humidity_2m = next(filter(lambda x: x.Variable() == Variable.relative_humidity and x.Altitude() == 2, current_variables))

print(f"Current time {current.Time()}")
print(f"Current temperature_2m {current_temperature_2m.Value()}")
print(f"Current relative_humidity_2m {current_relative_humidity_2m.Value()}")

Coordinates 52.540000915527344°N 13.419998168945312°E
Elevation 60.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Current time 1744160400
Current temperature_2m 7.050000190734863
Current relative_humidity_2m 52.0


In [7]:
def get_weather_at_timestamp(lat, lon, date_time):
    # Initialize Open-Meteo client
    om = openmeteo_requests.Client()
    
    # Define API parameters
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ["temperature_2m", "precipitation", "wind_speed_10m", "relative_humidity_2m"],  # Add more variables here if needed
        "timezone": "auto"
    }

    # Fetch data from Open-Meteo API
    responses = om.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
    response = responses[0]  # Extract first response

    # Extract metadata
    weather_info = {
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "elevation": response.Elevation(),
        "timezone": response.Timezone(),
        "timezone_abbreviation": response.TimezoneAbbreviation(),
        "utc_offset_seconds": response.UtcOffsetSeconds(),
    }

    # Extract hourly weather data
    hourly = response.Hourly()
    times = pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    )

    # Retrieve values for temperature and precipitation
    temperature_values = hourly.Variables(0).ValuesAsNumpy()
    precipitation_values = hourly.Variables(1).ValuesAsNumpy()
    wind_speed_values = hourly.Variables(2).ValuesAsNumpy()
    relative_humidity_values = hourly.Variables(3).ValuesAsNumpy()

    # Find the closest timestamp to the requested time
    closest_idx = min(range(len(times)), key=lambda i: abs(times[i] - date_time))

    weather_info.update({
        "time": times[closest_idx],
        "temperature_2m": temperature_values[closest_idx],
        "precipitation": precipitation_values[closest_idx],
        "wind_speed": wind_speed_values[closest_idx],
        "relative_humidity": relative_humidity_values[closest_idx]
    })

    return weather_info


In [8]:
pd.to_datetime(dfs[0]['Time'].iloc[0])

Timestamp('2023-12-25 19:01:31+0000', tz='UTC')

In [9]:
lat, lon = dfs[0]['Latitude'].iloc[0], dfs[0]['Longitude'].iloc[0]
timestamp = dfs[0]['Time'].iloc[0]

weather_data = get_weather_at_timestamp(lat, lon, pd.to_datetime(timestamp))
print(weather_data)

{'latitude': 33.18727493286133, 'longitude': -96.73574829101562, 'elevation': 229.0, 'timezone': b'America/Chicago', 'timezone_abbreviation': b'GMT-5', 'utc_offset_seconds': -18000, 'time': Timestamp('2025-04-08 05:00:00+0000', tz='UTC'), 'temperature_2m': 9.1, 'precipitation': 0.0, 'wind_speed': 6.8777895, 'relative_humidity': 51.0}


In [10]:
from retrying import retry
def convert_time_to_seconds(df):
    df['Time'] = pd.to_datetime(df['Time'])

    time_diff = df['Time'] - df['Time'].iloc[0]

    df['Time'] = time_diff.dt.total_seconds()

    return df

@retry(stop_max_attempt_number=3, wait_fixed=5000)  # Retry 3 times with a 5-second wait between retries
def fetch_hourly_weather(lat, lon):
    """
    Fetch hourly weather data for a given latitude and longitude.
    Returns a DataFrame with weather values for each hour.
    """
    # Initialize Open-Meteo client
    om = openmeteo_requests.Client()
    
    # Define API parameters
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ["temperature_2m", "precipitation", "wind_speed_10m", "relative_humidity_2m"],
        "timezone": "auto"
    }

    try:
        # Fetch data from Open-Meteo API with a longer timeout (e.g., 60 seconds)
        responses = om.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
        response = responses[0]  # Extract first response

        # Extract hourly weather data
        hourly = response.Hourly()
        times = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )

        # Retrieve values for each weather variable
        weather_data = pd.DataFrame({
            "time": times,
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "precipitation": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(3).ValuesAsNumpy()
        })

        return weather_data

    except requests.exceptions.Timeout:
        print("Request timed out. Please try again later.")
        return pd.DataFrame()  # Return an empty DataFrame in case of timeout


def get_weather_for_dataframe(df, lat, lon, time_column='Time'):
    # Fetch weather data for the given latitude and longitude
    weather_data = fetch_hourly_weather(lat, lon)

    # Ensure both dataframes' time columns are in the same time zone (UTC) and in the same format
    df[time_column] = pd.to_datetime(df[time_column])  # Ensure the column is in datetime format

    if df[time_column].dt.tz is None:  # If timezone is not set
        df[time_column] = df[time_column].dt.tz_localize('UTC')  # Localize to UTC
    else:  # If timezone is already set
        df[time_column] = df[time_column].dt.tz_convert('UTC')  # Convert to UTC

    # Check the column names in weather_data to confirm the correct time column name

    # Adjust this line if the time column is named differently
    if 'time' not in weather_data.columns:
        raise KeyError("'time' column is missing in the weather data")

    # Ensure weather data time is in UTC
    weather_data['time'] = pd.to_datetime(weather_data['time'], utc=True)  # Ensure time is in UTC

    # Merge weather data with the existing dataframe based on time
    merged_df = pd.merge_asof(df.sort_values(by=time_column),
                              weather_data.sort_values(by='time'),
                              left_on=time_column, right_on='time',
                              direction='nearest')

    # Return the dataframe with the weather features included
    return merged_df



In [11]:
og_df = dfs[0]

In [12]:
og_df.head()

,Latitude,Longitude,Elevation,Time,Cadence,Heart Rate
0,33.188945,-96.746148,229.000000,2023-12-25 19:01:31+00:00,80,114
1,33.189405,-96.739223,225.000000,2023-12-25 19:26:18+00:00,79,181
2,33.189435,-96.739331,225.000000,2023-12-25 19:26:19+00:00,80,182
3,33.189447,-96.739593,225.600006,2023-12-25 19:26:23+00:00,79,182
4,33.189432,-96.739762,226.000000,2023-12-25 19:26:29+00:00,80,182


In [15]:
dfs[0] = og_df

In [16]:
i = 1
for index in range(0,len(dfs)):
    dfs[index]['Time'] = pd.to_datetime(dfs[index]['Time'])

    # Update the dataframe in the list
    dfs[index] = get_weather_for_dataframe(
        dfs[index], 
        dfs[index]["Latitude"].iloc[0], 
        dfs[index]['Longitude'].iloc[0], 
        time_column='Time'
    )

    convert_time_to_seconds(dfs[index])
    dfs[index]['Altitude_diff'] = dfs[index]['Elevation'].diff()

    print(i)
    dfs[index].to_csv('./ReeceWeatherCSVs/weather_'+str(i)+'.csv', index=False)
    i += 1
    time.sleep(3)

dfs[0].head()



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113


,Latitude,Longitude,Elevation,Time,Cadence,Heart Rate,time,temperature_2m,precipitation,wind_speed_10m,relative_humidity_2m,Altitude_diff
0,33.188945,-96.746148,229.000000,0.0,80,114,2025-04-08 05:00:00+00:00,9.1,0.0,6.877789,51.0,NaN
1,33.189405,-96.739223,225.000000,1487.0,79,181,2025-04-08 05:00:00+00:00,9.1,0.0,6.877789,51.0,-4.000000
2,33.189435,-96.739331,225.000000,1488.0,80,182,2025-04-08 05:00:00+00:00,9.1,0.0,6.877789,51.0,0.000000
3,33.189447,-96.739593,225.600006,1492.0,79,182,2025-04-08 05:00:00+00:00,9.1,0.0,6.877789,51.0,0.600006
4,33.189432,-96.739762,226.000000,1498.0,80,182,2025-04-08 05:00:00+00:00,9.1,0.0,6.877789,51.0,0.399994


In [17]:
dfs[2].head()

,Latitude,Longitude,Elevation,Time,Cadence,Heart Rate,time,temperature_2m,precipitation,wind_speed_10m,relative_humidity_2m,Altitude_diff
0,30.365600,-95.546594,66.000000,0.0,48,84,2025-04-08 05:00:00+00:00,9.45,0.0,4.334974,62.0,NaN
1,30.365556,-95.546501,66.000000,4.0,85,84,2025-04-08 05:00:00+00:00,9.45,0.0,4.334974,62.0,0.000000
2,30.365494,-95.546395,66.000000,7.0,84,81,2025-04-08 05:00:00+00:00,9.45,0.0,4.334974,62.0,0.000000
3,30.365426,-95.546277,66.199997,10.0,84,87,2025-04-08 05:00:00+00:00,9.45,0.0,4.334974,62.0,0.199997
4,30.365401,-95.546237,66.199997,11.0,84,90,2025-04-08 05:00:00+00:00,9.45,0.0,4.334974,62.0,0.000000


In [18]:
dfs[45].tail()

,Latitude,Longitude,Elevation,Time,Cadence,Heart Rate,time,temperature_2m,precipitation,wind_speed_10m,relative_humidity_2m,Altitude_diff
432,33.190178,-96.741683,227.0,1800.0,80,197,2025-04-08 05:00:00+00:00,9.113,0.0,6.877789,51.0,0.0
433,33.190305,-96.741643,227.0,1803.0,80,198,2025-04-08 05:00:00+00:00,9.113,0.0,6.877789,51.0,0.0
434,33.190452,-96.741595,227.0,1807.0,80,198,2025-04-08 05:00:00+00:00,9.113,0.0,6.877789,51.0,0.0
435,33.190620,-96.741516,227.0,1812.0,80,199,2025-04-08 05:00:00+00:00,9.113,0.0,6.877789,51.0,0.0
436,33.190655,-96.741491,227.0,1813.0,80,199,2025-04-08 05:00:00+00:00,9.113,0.0,6.877789,51.0,0.0
